In [1]:
import math
import numpy as np
import pandas as pd
validator = "CitadelOne.csv"
path = f"./{validator}"
df_data= pd.read_csv(path)

In [ ]:
df_data['block_len'][1:].mean()

In [401]:
# df_data

In [3]:
total_missed = 0
indices = df_data['block_num']
for i in range(1,len(indices)):
    if int(indices[i]) != int(indices[i-1])+1:
        total_missed += 1
        print(i)

2
3
141
142
143
147


In [4]:
print(total_missed)
print(len(df_data))

6
200


In [404]:
df_data.reset_index(inplace=True)

In [405]:
print(df_data)

     index  block_num  Block_length_target  Goal_Bonded  \
0        0   12716389             7.233028       0.6666   
1        1   12716390             7.233028       0.6666   
2        2   12716391             7.233028       0.6666   
3        3   12716392             7.233028       0.6666   
4        4   12716393             7.233028       0.6666   
..     ...        ...                  ...          ...   
96      96   12716485             7.233028       0.6666   
97      97   12716486             7.233028       0.6666   
98      98   12716487             7.233028       0.6666   
99      99   12716488             7.233028       0.6666   
100    100   12716489             7.233028       0.6666   

     Inflation_Rate_Change  Min_Inflation_Rate  Max_Inflation_Rate  \
0                     0.13                0.07                 0.2   
1                     0.13                0.07                 0.2   
2                     0.13                0.07                 0.2   
3          

In [406]:
df_output = pd.DataFrame()
df_output['block Number'] = df_data.block_num
df_output['Atom Staked'] = df_data.atom_staked_v
df_output['Total_Rewards'] = ( df_data.commission_amt/(df_data.v_commission/100)) * 1E-6
df_output['Epoch Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Realized APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Expected APY'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)
df_output['Absolute Difference in APY'] = df_output.apply(lambda _: '', axis=1)
df_output['% Difference in Rewards'] = df_output.apply(lambda _: '', axis=1)

print(df_output.head())

   block Number   Atom Staked  Total_Rewards Epoch Rewards Expected Rewards  \
0      12716389  1.566610e+07    8419.437831                                  
1      12716390  1.566610e+07    8420.115536                                  
2      12716391  1.566610e+07    8421.242600                                  
3      12716392  1.566610e+07    8421.920067                                  
4      12716393  1.566610e+07    8422.599083                                  

  Realized APY Expected APY Absolute Difference in Rewards  \
0                                                            
1                                                            
2                                                            
3                                                            
4                                                            

  Absolute Difference in APY % Difference in Rewards  
0                                                     
1                                         

In [407]:
# Constants from Network


# Parameters to be varied by validator
for i in range(1,len(df_data)):

  Goal_Bonded                 = float(df_data.Goal_Bonded[i]) 
  Inflation_Rate_Change       = float(df_data.Inflation_Rate_Change[i]) 
  Min_Inflation_Rate          = float(df_data.Min_Inflation_Rate[i]) 
  Max_Inflation_Rate          = float(df_data.Max_Inflation_Rate[i]) 
  Block_length                = float(df_data.block_len[i]) 
  E_Block_length              = float(df_data.Block_length_target[i])
  Blocks_per_year             = float(df_data.Blocks_per_year[i]) 
  minProposerBonus            = float(df_data.min_proposer_bonus[i]) 
  maxProposerBonus            = float(df_data.max_proposer_bonus[i]) 
  Community_Tax               = float(df_data.community_tax[i]) 
    
  Min_Signatures              = float(df_data.Min_Signatures[i]) 
  Total_Fees_Per_Block        = float(df_data.total_block_fees[i]) * 1E-6
  Bonded_Ratio                = float(df_data.percent_staked[i]) /100
  stake                       = float(df_data.atom_staked_v[i]) 
  signatures_ratio            = float(df_data.sign_ratio[i]) 
  E_Inflation_Rate_BH         = float(df_data.inflation_rate[i]) 
  totalSupply                 = float(df_data.total_supply[i]) * 1E-6

  # E_current_Inflation_rate_change = (1 - Bonded_Ratio/Goal_Bonded) * Inflation_Rate_Change
  # E_Inflation_Rate_BH             = Prev_Inflation_Rate + E_current_Inflation_rate_change / Blocks_per_year

    # Check the new inflation rate is between min and max inflation rate
  if E_Inflation_Rate_BH > Max_Inflation_Rate:
    E_Inflation_Rate_BH = Max_Inflation_Rate
  elif E_Inflation_Rate_BH < Min_Inflation_Rate:
    E_Inflation_Rate_BH = Min_Inflation_Rate

  # Real_Inflation_Rate = E_Inflation_Rate_BH * Block_length/ E_Block_length

  E_validator         = stake / (totalSupply*Bonded_Ratio) * ((1+E_Inflation_Rate_BH)**(1/Blocks_per_year)-1) * totalSupply    # technically can cancel totalSupply but left for ease of understanding
  #E_validator         = stake / (totalSupply*Bonded_Ratio) * E_Inflation_Rate_BH/Blocks_per_year * totalSupply
  E_Proposer_Bonus                     = minProposerBonus + (signatures_ratio-Min_Signatures) / (1-Min_Signatures) * (maxProposerBonus-minProposerBonus)      
  E_Proposer_stake                     = totalSupply*Bonded_Ratio/175   # THIS IS POOR ESTIMATE AS VALIDATORS WITH MORE STAKE ARE SELECTED MORE FREQUENTLY, WOULD NEED ALL VALIDATOR'S DATA TO IMPROVE MODEL
  E_transaction_reward_per_atom_staked = Total_Fees_Per_Block * (1 - Community_Tax) / (totalSupply*Bonded_Ratio + E_Proposer_stake*E_Proposer_Bonus)

  E_ProposerTxFees                     = stake * (1 + E_Proposer_Bonus) * E_transaction_reward_per_atom_staked
  E_AttesterTxFees                     = stake * E_transaction_reward_per_atom_staked

  E_Attester = E_validator + E_AttesterTxFees
  E_Proposer = E_validator + E_ProposerTxFees

  P_Proposer = stake/totalSupply    # think this is true even with round robin selection
  P_attester = 1 - P_Proposer


  E_rewards = E_Proposer*P_Proposer + E_Attester*P_attester
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
  df_output['Expected Rewards'][i] = E_rewards
  df_output['Absolute Difference in Rewards'][i] = df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]
  df_output['% Difference in Rewards'][i] = (df_output['Expected Rewards'][i] - df_output['Epoch Rewards'][i]) / df_output['Epoch Rewards'][i]*100

  APY                                            = E_rewards/stake*Blocks_per_year*E_Block_length/Block_length
  Realized_APY                                   = df_output['Epoch Rewards'][i]/stake*Blocks_per_year*E_Block_length/Block_length
  df_output['Expected APY'][i]                   = APY*100
  df_output['Realized APY'][i]                   = Realized_APY*100     
  df_output['Absolute Difference in Rewards'][i] = E_rewards - df_output['Epoch Rewards'][i]
  df_output['Absolute Difference in APY'][i]     = (df_output['Expected APY'][i]  -   df_output['Realized APY'][i])

print(df_output)

     block Number   Atom Staked  Total_Rewards Epoch Rewards Expected Rewards  \
0        12716389  1.566610e+07    8419.437831                                  
1        12716390  1.566610e+07    8420.115536      0.677705         0.686862   
2        12716391  1.566610e+07    8421.242600      1.127063         0.686862   
3        12716392  1.566610e+07    8421.920067      0.677468         0.686862   
4        12716393  1.566610e+07    8422.599083      0.679015         0.686862   
..            ...           ...            ...           ...              ...   
96       12716485  1.566610e+07    8488.285278      0.677644         0.687284   
97       12716486  1.566610e+07    8488.962711      0.677433         0.687767   
98       12716487  1.566610e+07    8489.640431      0.677721         0.687077   
99       12716488  1.566610e+07    8490.317501       0.67707         0.686866   
100      12716489  1.566610e+07    8490.995017      0.677516         0.687303   

    Realized APY Expected A

/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_19076/160660359.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Epoch Rewards'][i]       = df_output.Total_Rewards[i]-df_output.Total_Rewards[i-1]
/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_19076/160660359.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_output['Expected Rewards'][i] = E_rewards
/var/folders/0y/dd7nd74d5ybdsz_l9kpznc300000gn/T/ipykernel_19076/160660359.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyda

In [408]:
# df_output.to_csv(f"drive/Shared drives/Model/Data Extraction/COSMOS/single-validators/Output Files/COSMOSValidationOutput-{validator}")

In [409]:
total_rewards = df_output['Epoch Rewards'][1:].sum()
expected_total = df_output['Expected Rewards'][1:].sum()

In [410]:
acual_total_rew_without_processing = (df_data.iloc[len(df_data)-1]['commission_amt']-df_data.iloc[0]['commission_amt'])/1e6/0.05
(acual_total_rew_without_processing-expected_total)/expected_total*100

320.79150200654595

In [411]:
expected_total

68.0215130773832

In [412]:
acual_total_rew_without_processing

286.2287465658998

In [413]:
(total_rewards-expected_total)/total_rewards*100

4.94104607802338

In [414]:
df_output[1:][df_output['Epoch Rewards'][1:]>0.010]

,block Number,Atom Staked,Total_Rewards,Epoch Rewards,Expected Rewards,Realized APY,Expected APY,Absolute Difference in Rewards,Absolute Difference in APY,% Difference in Rewards
1,12716390,1.566610e+07,8420.115536,0.677705,0.686862,21.572222,21.863695,0.009157,0.291473,1.351149
2,12716391,1.566610e+07,8421.242600,1.127063,0.686862,34.818745,21.219462,-0.440201,-13.599283,-39.057361
3,12716392,1.566610e+07,8421.920067,0.677468,0.686862,55.056478,55.819952,0.009395,0.763474,1.38671
4,12716393,1.566610e+07,8422.599083,0.679015,0.686862,21.454467,21.702406,0.007847,0.247939,1.155652
5,12716394,1.566610e+07,8423.277121,0.678039,0.686862,21.082767,21.357126,0.008824,0.27436,1.301345
...,...,...,...,...,...,...,...,...,...,...
96,12716485,1.566610e+07,8488.285278,0.677644,0.687284,21.597591,21.904836,0.00964,0.307245,1.422591
97,12716486,1.566610e+07,8488.962711,0.677433,0.687767,22.055285,22.391738,0.010334,0.336453,1.5255
98,12716487,1.566610e+07,8489.640431,0.677721,0.687077,19.892946,20.167579,0.009356,0.274633,1.380553
99,12716488,1.566610e+07,8490.317501,0.67707,0.686866,21.565639,21.877657,0.009796,0.312018,1.446829


In [415]:
df_output

,block Number,Atom Staked,Total_Rewards,Epoch Rewards,Expected Rewards,Realized APY,Expected APY,Absolute Difference in Rewards,Absolute Difference in APY,% Difference in Rewards
0,12716389,1.566610e+07,8419.437831,,,,,,,
1,12716390,1.566610e+07,8420.115536,0.677705,0.686862,21.572222,21.863695,0.009157,0.291473,1.351149
2,12716391,1.566610e+07,8421.242600,1.127063,0.686862,34.818745,21.219462,-0.440201,-13.599283,-39.057361
3,12716392,1.566610e+07,8421.920067,0.677468,0.686862,55.056478,55.819952,0.009395,0.763474,1.38671
4,12716393,1.566610e+07,8422.599083,0.679015,0.686862,21.454467,21.702406,0.007847,0.247939,1.155652
...,...,...,...,...,...,...,...,...,...,...
96,12716485,1.566610e+07,8488.285278,0.677644,0.687284,21.597591,21.904836,0.00964,0.307245,1.422591
97,12716486,1.566610e+07,8488.962711,0.677433,0.687767,22.055285,22.391738,0.010334,0.336453,1.5255
98,12716487,1.566610e+07,8489.640431,0.677721,0.687077,19.892946,20.167579,0.009356,0.274633,1.380553
99,12716488,1.566610e+07,8490.317501,0.67707,0.686866,21.565639,21.877657,0.009796,0.312018,1.446829
